In [19]:
using LinearAlgebra
using Plots
using Statistics

In [20]:
# Generate the Hankel, assuming z is a stacked vector with block dimension d
function Hankel(z::Array{Float64}, d::Int64, L::Int64)
    N = Int64(round(length(z) / d))
    @assert length(z) == N * d
    z = reshape(z, (d, N))
    H = Matrix{Float64}(zeros(d * L, N - L + 1))
    for i = 1:(N - L + 1); H[:, i] = (z[:, i:(i + L - 1)])[:] end
    return H
end
;

In [21]:
# For a particular value of N, calculate the statistics of averaging N different Hw's
# Generate M (=1000) different norm(Hw) to get the mean, delta, and (1-delta) quantiles
function sampleTrueN(n::Int64, sigma::Float64, T::Int64, L::Int64, N::Int64, delta::Float64; M=1000)
    norms = zeros(M)
    for m = 1:M
        w = (sigma / sqrt(N)) * randn(n * T)
        Hw = Hankel(w, n, L)
        norms[m] = opnorm(Hw)
    end
    # Mean
    mu = mean(norms)
    
    # delta quantile
    qlonorm = quantile(norms, delta)
    
    # (1-delta) quantile
    qhinorm = quantile(norms, 1 - delta)
    
    return (mu, qlonorm, qhinorm)
end


# For a particular N, calculate the statistics of 
# Run a bootstrap experiment:
# Over M (=1) iterations, get call sampleTrueN to figure out the bootstrap epsilon
# The bootstrap epsilon is the (1-delta) quantile of the empirical true H_L (w)
function sampleBootN(n::Int64, sigma::Float64, T::Int64, L::Int64, N::Int64, delta::Float64; M=1)
    Bs = zeros(M)
    for m = 1:M
        W = sigma * randn((n * T), N)
        (mu, qlo, qhi) = sampleTrueN(n, sigma, T, L, N, delta)
        Bs[m] = qhi
    end
    
    # Mean of boot eps
    muB = mean(Bs)
    
    # delta quantile of boot eps
    qloB = quantile(Bs, delta)
    
    # (1-delta) quantile of boot eps
    qhiB = quantile(Bs, 1 - delta)
    
    return (muB, qloB, qhiB)
end

;

In [22]:
# Slightly unstable graph Laplacian is dim n = 3
n = 3
sigma = sqrt(0.1)

# Hankel matrix dimensions
T = 45
L = 10

# Uncertainty parameter delta
delta = 0.05

# 10, 20, 30, 40, ..., 10000
Ns = 10:10:10000

println("Generating trues")
# About 150 secs
@time trues = [sampleTrueN(n, sigma, T, L, k, delta) for k in Ns]

println("Generating boots")
# About 150 secs
@time boots = [sampleBootN(n, sigma, T, L, k, delta) for k in Ns]
;

Generating trues
145.344276 seconds (86.10 M allocations: 68.304 GiB, 3.59% gc time)
Generating boots
140.486977 seconds (86.09 M allocations: 78.372 GiB, 4.47% gc time)


In [23]:
NN = length(Ns)
# Mean of true ||Hw||
muTs = [trues[i][1] for i = 1:NN]

# delta quantile of true ||Hw||
qloTs = [trues[i][2] for i = 1:NN]

# (1-delta) quantile of true ||Hw||
qhiTs = [trues[i][3] for i = 1:NN]

# Mean of boot eps
muBs = [boots[i][1] for i = 1:NN]

# We don't actually plot these two
qloBs = [boots[i][2] for i = 1:NN]
qhiBs = [boots[i][3] for i = 1:NN]
;

In [31]:
# Plot the true ||Hw|| data, along with ribbons
plot(Ns, muTs,
    ribbon=(muTs - qloTs, qhiTs - muTs),
    fillalpha=0.1,
    xticks=([10; 31; 100; 316; 1000; 3162; 10000] , ["1.0"; "1.5"; "2.0"; "2.5"; "3.0"; "3.5"; "4.0";]),
#     xticks=([2000; 6000; 10000]),
    label="true ||H_L (w)||",
    xlabel="Number of Samples (log10 N)",
    ylabel="Operator Norm",
    xscale=:log10,
    yscale=:log10,
    xtickfontsize=18,
    ytickfontsize=18,
    legendfontsize=18,
    xguidefontsize=18,
    yguidefontsize=18
)


# Plot the solid eps-B line
plot!(Ns, muBs,
    label="ε Boot"
)

# Check which directory you booted jupyter notebook from
savefig("Hwnorms.png")


In [32]:
# The particular values we will use in the controller synthesis
[
    sampleBootN(n, sigma, T, L, 1, delta)[1];
    sampleBootN(n, sigma, T, L, 3, delta)[1];
    sampleBootN(n, sigma, T, L, 10, delta)[1];
    sampleBootN(n, sigma, T, L, 31, delta)[1];
    sampleBootN(n, sigma, T, L, 100, delta)[1];
    sampleBootN(n, sigma, T, L, 316, delta)[1];
    sampleBootN(n, sigma, T, L, 1000, delta)[1];
    sampleBootN(n, sigma, T, L, 3162, delta)[1];
    sampleBootN(n, sigma, T, L, 10000, delta)[1];
]

9-element Array{Float64,1}:
 3.9082042192458024 
 2.299229011763075  
 1.2477926488188928 
 0.7080045365205088 
 0.40338514816909393
 0.22333114401705384
 0.12494235476552938
 0.07121620824611384
 0.03917996646748791